In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
from sympy import *
import plotly.offline as py
import plotly.graph_objs as go
%matplotlib inline
init_printing()

In [2]:
Variables=symbols("x1 y1 x2 y2")
x1,y1,x2,y2 = Variables
Time=symbols("t")
t=Time
Parameters = symbols("alpha beta gamma delta w a0 a1")
alpha,beta,gamma,delta,w,a0,a1 = Parameters

### Duffing system

In [3]:
F=Matrix([y1, gamma*cos(w*t) - delta*y1 - beta*x1 - alpha*x1**3,
          y2, gamma*cos(w*t) - delta*y2 - beta*x2 - alpha*x2**3])
F

⎡                y₁                ⎤
⎢                                  ⎥
⎢      3                           ⎥
⎢- α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(t⋅w)⎥
⎢                                  ⎥
⎢                y₂                ⎥
⎢                                  ⎥
⎢      3                           ⎥
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w)⎦

In [4]:
zeta=Matrix([0,0,
             0,0])
#zeta=eps*zeta
zeta

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

### Synchrony

In [5]:
phi=Matrix([(x2-x1*(a0+a1*sin(w*t))),y2-y1*(a0+a1*sin(w*t))])
phi

⎡-x₁⋅(a₀ + a₁⋅sin(t⋅w)) + x₂⎤
⎢                           ⎥
⎣-y₁⋅(a₀ + a₁⋅sin(t⋅w)) + y₂⎦

In [6]:
N=simulation.VectorDivergence(phi,Variables)
N

⎡-a₀ - a₁⋅sin(t⋅w)          0          1  0⎤
⎢                                          ⎥
⎣        0          -a₀ - a₁⋅sin(t⋅w)  0  1⎦

In [7]:
on_manifold = [(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t)))]

## Invariance

In [8]:
# NF=(N*(F+zeta)).subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
# simplify(NF)
NF=N*(F+zeta)
NF
#simplify(NF)

⎡                                 y₁⋅(-a₀ - a₁⋅sin(t⋅w)) + y₂                 
⎢                                                                             
⎢      3                                                  ⎛      3            
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w) + (-a₀ - a₁⋅sin(t⋅w))⋅⎝- α⋅x₁  - β⋅x₁ - δ⋅

                ⎤
                ⎥
               ⎞⎥
y₁ + γ⋅cos(t⋅w)⎠⎦

In [9]:
dPhi=diff(phi,Time)
dPhi

⎡-a₁⋅w⋅x₁⋅cos(t⋅w)⎤
⎢                 ⎥
⎣-a₁⋅w⋅y₁⋅cos(t⋅w)⎦

In [10]:
eqn=(NF+dPhi)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(eqn)

⎡                               -a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(a₀ + a₁⋅sin(t⋅w)) + y₂
⎢                                                                             
⎢                        3                                                 ⎛  
⎣-a₁⋅w⋅y₁⋅cos(t⋅w) - α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w) + (a₀ + a₁⋅sin(t⋅w))⋅⎝α⋅

                               ⎤
                               ⎥
  3                           ⎞⎥
x₁  + β⋅x₁ + δ⋅y₁ - γ⋅cos(t⋅w)⎠⎦

In [11]:
# This is working only because of the nature of the N matrix - be careful
zeta[2]=-(eqn)[0]
zeta[3]=-(eqn)[1]

simplify(zeta)

⎡                                                     0                       
⎢                                                                             
⎢                                                     0                       
⎢                                                                             
⎢                               a₁⋅w⋅x₁⋅cos(t⋅w) + y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂ 
⎢                                                                             
⎢                       3                                                 ⎛   
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) - (a₀ + a₁⋅sin(t⋅w))⋅⎝α⋅x

                              ⎤
                              ⎥
                              ⎥
                              ⎥
                              ⎥
                              ⎥
 3                           ⎞⎥
₁  + β⋅x₁ + δ⋅y₁ - γ⋅cos(t⋅w)⎠⎦

In [14]:
NF=N*(F+zeta)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(NF)
simplify(NF+dPhi).subs(on_manifold)

⎡0⎤
⎢ ⎥
⎣0⎦

In [19]:
NZ=N*zeta
NZ

⎡                                a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(-a₀ - a₁⋅sin(t⋅w)) - y
⎢                                                                             
⎢                       3                                                  ⎛  
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) - (-a₀ - a₁⋅sin(t⋅w))⋅⎝- 

₂                                ⎤
                                 ⎥
    3                           ⎞⎥
α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(t⋅w)⎠⎦

## Stability

In [20]:
chi=Matrix([0,0,
            -(x2-x1*(a0+a1*sin(w*t))),-(y2-y1*(a0+a1*sin(w*t)))])

#chi=(epsilon1*sin(b*t)+epsilon2)*chi 
#chi=epsilon*chi
#epsilon=1
chi

⎡            0             ⎤
⎢                          ⎥
⎢            0             ⎥
⎢                          ⎥
⎢x₁⋅(a₀ + a₁⋅sin(t⋅w)) - x₂⎥
⎢                          ⎥
⎣y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂⎦

In [21]:
del_zeta=simulation.VectorDivergence(zeta,Variables)
del_zeta

⎡                0                                    0                       
⎢                                                                             
⎢                0                                    0                       
⎢                                                                             
⎢          a₁⋅w⋅cos(t⋅w)                       a₀ + a₁⋅sin(t⋅w)               
⎢                                                                             
⎢                   ⎛        2    ⎞                                           
⎣(a₀ + a₁⋅sin(t⋅w))⋅⎝- 3⋅α⋅x₁  - β⎠  a₁⋅w⋅cos(t⋅w) - δ⋅(a₀ + a₁⋅sin(t⋅w))  3⋅α

  0       0 ⎤
            ⎥
  0       0 ⎥
            ⎥
  0       -1⎥
            ⎥
   2        ⎥
⋅x₂  + β  δ ⎦

In [22]:
del_chi=simulation.VectorDivergence(chi,Variables)
del_chi

⎡       0                 0          0   0 ⎤
⎢                                          ⎥
⎢       0                 0          0   0 ⎥
⎢                                          ⎥
⎢a₀ + a₁⋅sin(t⋅w)         0          -1  0 ⎥
⎢                                          ⎥
⎣       0          a₀ + a₁⋅sin(t⋅w)  0   -1⎦

In [23]:
del_F=simulation.VectorDivergence(F,Variables)
del_F

⎡      0        1         0        0 ⎤
⎢                                    ⎥
⎢        2                           ⎥
⎢- 3⋅α⋅x₁  - β  -δ        0        0 ⎥
⎢                                    ⎥
⎢      0        0         0        1 ⎥
⎢                                    ⎥
⎢                           2        ⎥
⎣      0        0   - 3⋅α⋅x₂  - β  -δ⎦

In [24]:
(del_F+del_zeta+del_chi)

⎡                0                                              1             
⎢                                                                             
⎢                  2                                                          
⎢          - 3⋅α⋅x₁  - β                                       -δ             
⎢                                                                             
⎢ a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w)                      a₀ + a₁⋅sin(t⋅w)      
⎢                                                                             
⎢                   ⎛        2    ⎞                                           
⎣(a₀ + a₁⋅sin(t⋅w))⋅⎝- 3⋅α⋅x₁  - β⎠  a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w) - δ⋅(a₀ 

                0   0 ⎤
                      ⎥
                      ⎥
                0   0 ⎥
                      ⎥
                -1  0 ⎥
                      ⎥
                      ⎥
+ a₁⋅sin(t⋅w))  0   -1⎦

In [25]:
diff(N,t)*N.transpose()

⎡-a₁⋅w⋅(-a₀ - a₁⋅sin(t⋅w))⋅cos(t⋅w)                  0                 ⎤
⎢                                                                      ⎥
⎣                0                   -a₁⋅w⋅(-a₀ - a₁⋅sin(t⋅w))⋅cos(t⋅w)⎦

In [38]:
Jac = N*(del_F+del_zeta+del_chi)*N.transpose() + diff(N,t)*N.transpose()
Jac = simplify(Jac)
Jac.subs(on_manifold)

⎡    2                        2    2                                          
⎢- a₀  - 2⋅a₀⋅a₁⋅sin(t⋅w) - a₁ ⋅sin (t⋅w) - 1                       0         
⎢                                                                             
⎢                                                  2                        2 
⎣                     0                        - a₀  - 2⋅a₀⋅a₁⋅sin(t⋅w) - a₁ ⋅

             ⎤
             ⎥
             ⎥
   2         ⎥
sin (t⋅w) - 1⎦

In [39]:
Jac.eigenvals()

### coupling

In [40]:
chi

⎡            0             ⎤
⎢                          ⎥
⎢            0             ⎥
⎢                          ⎥
⎢x₁⋅(a₀ + a₁⋅sin(t⋅w)) - x₂⎥
⎢                          ⎥
⎣y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂⎦

In [41]:
zeta

⎡                                                      0                      
⎢                                                                             
⎢                                                      0                      
⎢                                                                             
⎢                                a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(-a₀ - a₁⋅sin(t⋅w)) - y
⎢                                                                             
⎢                       3                                                  ⎛  
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) - (-a₀ - a₁⋅sin(t⋅w))⋅⎝- 

                                 ⎤
                                 ⎥
                                 ⎥
                                 ⎥
₂                                ⎥
                                 ⎥
    3                           ⎞⎥
α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(t⋅w)⎠⎦

In [42]:
simplify((chi+zeta))#.subs([(x2,a*t*x1),(y2,b*t*y1),(z2,c*t*z1)])))

⎡                                                                   0         
⎢                                                                             
⎢                                                                   0         
⎢                                                                             
⎢                               a₁⋅w⋅x₁⋅cos(t⋅w) + x₁⋅(a₀ + a₁⋅sin(t⋅w)) - x₂ 
⎢                                                                             
⎢                       3                                                     
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) + y₁⋅(a₀ + a₁⋅sin(t⋅w)) -

                                                           ⎤
                                                           ⎥
                                                           ⎥
                                                           ⎥
+ y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂                               ⎥
                                                           ⎥
 